#pycaretはtrain sampleとtest sampleを予め分けて準備し、5つのステップでtest sampleでの精度を解析する

①training dataとtest dataを読み込む

②狭義のtraining set (training sample全体の7割)を10分割交差検証で比較しハイパーパラメーターのチューニングする

③validation set (training sample全体の3割)も使用して、training sample全体でのモデルを作成する

④training set全体での精度の検証 (internal validation)

⑤予め別に自身で準備したtest sampleで精度を検証する (external validation) →この精度を上げるのが目的

In [ ]:
#パッケージのインストール
!pip install pycaret

In [ ]:
#ライブラリのインポート
import numpy as np
import pandas as pd

In [ ]:
#回帰パッケージのインポート
from pycaret.regression import *

#ここからは実際のデータの読み込み

In [ ]:
# ローカルファイルを直接アップロードする (train data)
from google.colab import files
uploaded = files.upload()

In [ ]:
#train dataの読み込み
train_data = pd.read_csv('train.csv')
train_data.head()

In [ ]:
# ローカルファイルを直接アップロードする (test data)
from google.colab import files
uploaded = files.upload()

In [ ]:
#test dataの読み込み
test_data = pd.read_csv('test.csv')
test_data.head()

#ここからステップ①、pycaretにデータの型を判別させ、training data全体をtrainingのための狭義のtraining sampleとパラメーターを決めるためのvalidation sampleに分割する。

In [ ]:
#train_sizeを指定しない場合は7:3に分割される
#pycaretにデータの型を判別させる
exp1 = setup(train_data, target = 'cancertime', train_size = 0.8,ignore_features = None)

#ここからステップ② 狭義のtraining set (上でtraining sample全体の8割に指定)を10分割交差検証で比較しハイパーパラメーターのチューニングする

In [ ]:
#transformed train sampleでのmodelの比較 (デフォルトは10分割交差検証)
compare_models()

In [ ]:
#lightGBM (Gradient Boosting Machine)を作成
lgbm = create_model('lightgbm')

In [ ]:
#Random Forest Regressorを作成
rf = create_model('rf')

In [ ]:
#Extra Trees Regressorを作成
et = create_model('et')

In [ ]:
#linear regressionを作成
lr = create_model('lr')

In [ ]:
#linear regressionを作成
lr = create_model('lr')

In [ ]:
#Lasso Regressionを作成
lasso = create_model('lasso')

In [ ]:
#Ridge Regressionを作成
ridge = create_model('ridge')

#ここからハイパーパラメーターをチューニング、pycaretはランダムサーチで最適パラメーターを探索

In [ ]:
#試行回数の指定 n_iter = 10 (時間がかかるので10にしています、50や100に設定してください)
#ここでは'RMSE'を最適なものにしている (他r2などを指定可能)
tuned_lgbm = tune_model(lgbm,  n_iter = 10, fold=10,optimize = 'RMSE')

In [ ]:
tuned_rf = tune_model(rf,  n_iter = 10, fold=10,optimize = 'RMSE')

In [ ]:
tuned_et = tune_model(et,  n_iter = 10, fold=10,optimize = 'RMSE')

In [ ]:
tuned_lr = tune_model(lr,  n_iter = 10, fold=10,optimize = 'RMSE')

In [ ]:
tuned_lasso = tune_model(lasso,  n_iter = 10, fold=10,optimize = 'RMSE')

In [ ]:
tuned_ridge = tune_model(ridge,  n_iter = 10, fold=10,optimize = 'RMSE')

#ここからステップ③ validation sampleも含めたtraining sample2100例全体でのモデル (final model)を固定する

In [ ]:
#モデルの固定
final_lgbm = finalize_model(tuned_lgbm)

In [ ]:
final_rf = finalize_model(tuned_rf)

In [ ]:
final_et = finalize_model(tuned_et)

In [ ]:
final_lr = finalize_model(tuned_lr)

In [ ]:
final_lasso = finalize_model(tuned_lasso)

In [ ]:
final_ridge = finalize_model(tuned_ridge)

#ここからステップ④　training set全体での精度検証 (internal validation)

このコードは厳密にはtraining sample全体で評価を行っているのではなく、training sampleの一部である (今回ならtraining sampleの2割)　validation sampleのみでの評価を行っている

→training sample全体で行うためには、本プロセスの最後で行っているtest dataでの評価の際に用いるtest dataをtraining dataに置き換えて、test dataに行うのと同様にScoreのCSVファイルを出力する

In [ ]:
#lightgbmモデルのinternal validation
predict_model(final_lgbm);

In [ ]:
predict_model(final_rf);

In [ ]:
predict_model(final_et);

In [ ]:
predict_model(final_lr);

In [ ]:
predict_model(final_lasso);

In [ ]:
predict_model(final_ridge);

#ここからステップ⑤　予め別に自身で準備したtest sampleで精度を検証する (external validation)→この精度を上げるのが目的

In [ ]:
#モデルの評価
prediction_lgbm = predict_model(final_lgbm, data = test_data)
prediction_lgbm

In [ ]:
prediction_rf = predict_model(final_rf, data = test_data)
prediction_rf

In [ ]:
prediction_et = predict_model(final_et, data = test_data)
prediction_et

In [ ]:
prediction_lr = predict_model(final_lr, data = test_data)
prediction_lr

In [ ]:
prediction_lasso = predict_model(final_lasso, data = test_data)
prediction_lasso

In [ ]:
prediction_ridge = predict_model(final_ridge, data = test_data)
prediction_ridge

#統計ソフトでの解析用に機械学習が出したスコアと正解ラベルをcsvファイルで出力する。

例えば、RMSEは予測値と真値の差を二乗してその平均を平方したものなので、エクセルの予測値と実際の値を用いて計算することもできる

In [ ]:
#モデルの予測出力 (左のフォルダにcsvが出ているので、左のファイルマークよりダウンロード)
prediction_lgbm.to_csv('prediction_lgbd.csv', columns=['cancertime','prediction_label'])

In [ ]:
prediction_rf.to_csv('prediction_rf.csv', columns=['cancertime','prediction_label'])

In [ ]:
prediction_et.to_csv('prediction_et.csv', columns=['cancertime','prediction_label'])

In [ ]:
prediction_lr.to_csv('prediction_lr.csv', columns=['cancertime','prediction_label'])

In [ ]:
prediction_lasso.to_csv('prediction_lasso.csv', columns=['cancertime','prediction_label'])

In [ ]:
prediction_ridge.to_csv('prediction_ridge.csv', columns=['cancertime','prediction_label'])

#ハイパーパラメーターや変数の重要度等を一斉に出力する(一部出力不可のモデルもある)

In [ ]:
evaluate_model (final_lgbm)

In [ ]:
evaluate_model (final_rf)

In [ ]:
evaluate_model (final_et)

In [ ]:
evaluate_model (final_lr)

In [ ]:
evaluate_model (final_lasso)

In [ ]:
evaluate_model (final_ridge)